# Imports


In [145]:
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, join

import pandas as pd
import urllib
import re
import glob
import sys
reload(sys)
sys.setdefaultencoding('utf-8')


# Functions

In [189]:

#Param soup object
#@returns a non duplicate list of information from span tags
def no_duplicates(s):
    c = 0
    with_duplicates = []
    for x in s.find_all('span'):
        with_duplicates.append(x.string)
        c = c + 1
    no_dups = []
    i = 0
    for item in with_duplicates:
        if item in no_dups:
            pass
        else:
            no_dups.append(item)
    
    return no_dups


#gets mobile phone number
#Param soup object
#@returns mobile, work, home and fax number
def get_mobile(s):
    n = no_duplicates(s)
    i = 0;
    mobile ='None'
    phone ='None'
    work ='None'
    home ='None'
    fax = 'None'
    for x in n:
        if (n[i] == 'mobile' or n[i] == 'phone') or (n[i] == 'Mobile' or n[i] == 'Phone') :
                if i + 1 >= len(n):
                    mobile = "None"
                else:
                    mobile = n[i + 1]   
                
        if n[i] == 'fax' or n[i] == 'Fax':
            fax = n[i + 1]
        if n[i] == 'work' or n[i] == 'Work':
            work = n[i + 1]
        if n[i] == 'home':
            home = n[i + 1]
               
        i = i + 1
   
    return mobile,work,home,fax

#get info contents from span and div tags
def span_ls():
    return no_duplicates()


#returns a stripped list of information in div tags
def div_ls(s):
    div_list = []

    for x in s.find_all('div'):
            div_list.append(x.string)
            
    div_list = [x for x in div_list if x != None]
    return div_list


#gets first name & last name and splits the string*****
#Param soup object
#returns first name and last name
def get_name(s):
    t = 0
    mylist= []
    contact_list = []

    for x in s.find_all('span'):
        mylist.append(x.string)
        t = t + 1
    clean = mylist[3]
    contact_list = re.split('\s+', clean)
    if len(contact_list) == 1:
        return contact_list[0]
    
        
    return contact_list[0], contact_list[1]



#param soup obejct
#@returns email
def get_email(s):
    e = 0
    ls = div_ls(s)
    em =[]
    for emails in ls:

        emails = str(emails)
        if '@' in emails:
            email = ls.__getitem__(e)
            #ls.remove(email)
            em.append(email + ' ') 
            return email
        e = e + 1    
    

#param soup obejct
#@returns address
def get_address(s):
    i = 0
    ls = div_ls(s)
    for address in ls:
        address = str(address)
        address = address.rstrip('\n')
        if len(address) > 40:
            return address
        i = i + 1
        
        
#param soup obejct
#@returns wesbite
def get_website(s):
    e = 0
    ls = div_ls(s)
    for x in ls:
        if x is 'None':
            pass
    
        elif 'www' in x:
            website = ls.__getitem__(e).strip()
            return website
        
        e = e + 1


# List of all file names from directory

In [198]:
#This section will get all file names to run through code

main_path = ""

#this puts all file names in list
#@returns list of all files names
def get_file_name_list():
    ls = []
    os.chdir(main_path)
    for file in glob.glob("*.html"):
        ls.append(file)
    return ls


# Builds Dataframe

In [193]:

#Param soup object
#@returns final dataframe
def build_data_frame(s):
    
    columns = ['First','Last', 'Email','Mobile','Work','Home','Fax','Website','Address']
    index = [0]
    f = pd.DataFrame(index=index, columns=columns)
    f = f.fillna('None') 
    f['First'] = get_name(s)[0]
    f['Last'] = get_name(s)[1]
    f['Email'] = get_email(s)
    f['Mobile'] = get_mobile(s)[0]
    f['Work'] = get_mobile(s)[1]
    f['Home'] = get_mobile(s)[2]
    f['Fax'] = get_mobile(s)[3]
    f['Website'] = get_website(s)
    f['Address'] = get_address(s)    
    
    return f

# Runs Program

In [194]:
path = ''

# runs program
def run():
    file_list = get_file_name_list()
    i = 0
    df = pd.DataFrame()
    for x in file_list:
        if len(df) is 0:
            main_path = path +file_list[i]
            soup = BeautifulSoup(open(main_path),"html5lib")
            df = build_data_frame(soup)
            i = i + 1
        else:
            main_path = path +file_list[i]
            soup = BeautifulSoup(open(main_path),"html5lib")
            df1 = build_data_frame(soup)
            df = pd.concat([df, df1], axis=0)
            i = i + 1
        
    return df
    
  


# Convert Dataframe to Excel Sheet

In [201]:
# DF TO EXCEL
from pandas import ExcelWriter

writer = ExcelWriter('evernote_contact_list.xlsx')
final.to_excel(writer, startcol = 0, startrow = 9)

worksheet = writer.sheets['Sheet1']
worksheet.write_string(0, 0, 'evernote contact list')

writer.save()